### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
8,38352559,123387,79085,51530,0,0,943,2,2021-08-02 22:00,2021-08-02 22:00,2,1,888,0,1
9,38353921,88711,79085,51530,0,0,943,3,2021-08-03 22:00,2021-08-03 22:01,6,0,6272,4,1
10,38351380,10516,79085,3,0,0,943,4,2021-08-04 21:59,2021-08-04 22:00,4,0,2832,2,1
11,38354731,31580,79085,51530,0,0,943,5,2021-08-05 18:00,2021-08-05 18:00,3,1,366,0,1
12,38354733,79085,31580,51530,0,0,943,6,2021-08-06 18:00,2021-08-06 18:00,1,5,1320,4,1
13,38354732,31580,79085,51530,0,0,944,0,2021-08-07 18:00,2021-08-07 18:00,6,0,448,3,1
14,38354350,79085,47773,3,0,0,944,1,2021-08-08 21:59,2021-08-08 21:59,3,0,1365,4,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

17

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,123387,30525,38320,2,15,2,0,0,5 4 1,7,8,9,38352559
1,79085,39797,32002,1,1,5,0,1,5-2-3b,6,6,6,38352559
0,88711,24967,38537,1,19,1,0,0,TriD2,9,7,8,38353921
1,79085,45008,31438,0,2,2,0,0,5-2-3b,6,6,6,38353921
0,10516,32466,39775,1,10,0,0,0,TORA TORA TORA,7,7,8,38351380
1,79085,37509,30200,0,1,0,0,0,5-2-3b,5,4,6,38351380
0,31580,29387,35792,2,12,3,2,0,GOAL,9,6,6,38354731
1,79085,40599,34194,6,1,3,1,0,5-2-3b,6,6,6,38354731
0,79085,48427,31583,5,1,1,0,0,5-2-3b,5,6,6,38354733
1,31580,21731,38575,2,13,0,0,0,GOAL,8,6,7,38354733


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

34

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2074

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,89684,0,0,0,0,0,0,0,8,2074
1,6335,0,0,0,0,0,0,0,7,2074
2,121325,0,0,0,0,0,0,0,6,2074
3,79085,0,0,0,0,0,0,0,5,2074
4,10467,0,0,0,0,0,0,0,4,2074
5,121670,0,0,0,0,0,0,0,3,2074
6,48360,0,0,0,0,0,0,0,2,2074
7,78774,0,0,0,0,0,0,0,1,2074


#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)

#query= 'leagueID not in @ids and teamID not in @team and round not in @rodada'

#df_league_stats=  df_league_stats.query(query)

# Adicionando informações se forem estatísticas ainda não adicionada
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [14]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [15]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [16]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [17]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

1

In [18]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()